In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly

In [ ]:
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import kraft

In [ ]:
np.random.seed(seed=kraft.RANDOM_SEED)

In [ ]:
genes = pd.read_csv("genes.txt", sep="\t", squeeze=True).sort_values().tolist()

len(genes)

In [ ]:
n_sample = 10

gene_x_sample = pd.DataFrame(
    np.random.normal(size=(len(genes), n_sample)),
    index=genes,
    columns=("Sample {}".format(i) for i in range(n_sample)),
)

gene_x_sample.shape

In [ ]:
n_gene_set = 50

gene_sets = []

for i in range(n_gene_set):

    gene_sets.append(np.random.choice(genes, size=i * 10, replace=False))

gene_sets = pd.DataFrame(
    gene_sets, index=("Gene Set {}".format(i) for i in range(n_gene_set))
)

gene_sets.shape

In [ ]:
gene_score_no_na_sorted = gene_x_sample.iloc[:, 0].dropna().sort_values(ascending=False)

In [ ]:
gene_set_genes = gene_sets.iloc[0]

kraft.run_single_sample_gsea(gene_score_no_na_sorted, gene_set_genes)

In [ ]:
gene_set_genes = gene_sets.iloc[1]

for normalization_method in ("-0-", "0-1"):

    if normalization_method is None:

        gene_score_no_na_sorted_ = gene_score_no_na_sorted

    else:

        gene_score_no_na_sorted_ = kraft.normalize_series_or_dataframe(
            gene_score_no_na_sorted, None, normalization_method
        )

    for statistic in ("ks", "auc"):

        kraft.run_single_sample_gsea(
            gene_score_no_na_sorted_,
            gene_set_genes,
            statistic=statistic,
            title="normalization_method: {}<br>statistic: {}".format(
                normalization_method, statistic
            ),
        )

In [ ]:
%%timeit

kraft.run_single_sample_gsea(gene_score_no_na_sorted, gene_set_genes, plot=False)

In [ ]:
import cProfile

In [ ]:
cProfile.run(
    "kraft.run_single_sample_gsea(gene_score_no_na_sorted, gene_set_genes, plot=False)",
    sort="time",
)

In [ ]:
kraft.run_single_sample_gseas(gene_x_sample.iloc[:, [0]], gene_sets)

In [ ]:
%%timeit

kraft.run_single_sample_gseas(gene_x_sample.iloc[:, [0]], gene_sets)

In [ ]:
cProfile.run(
    "kraft.run_single_sample_gseas(gene_x_sample.iloc[:, [0]], gene_sets)", sort="time"
)

In [ ]:
from scipy.stats import pearsonr


def function(x, y):

    return pearsonr(x, y)[0]

In [ ]:
n_sample_per_phenotype = n_sample // 2

phenotypes = np.array((0,) * n_sample_per_phenotype + (1,) * n_sample_per_phenotype)

kraft.run_gsea(
    gene_x_sample,
    phenotypes,
    gene_set_genes,
    function,
    statistic="ks",
    n_permutation=10,
    permuting="gene",
    gene_score_name="Correlation",
)